In [ ]:
import os
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine

In [ ]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

## Preping vector data

In [ ]:
# Define projection
repro_crs = 'epsg:4326'

# Load data and convert to equal area projection
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con).to_crs(repro_crs) 
flood_l1 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_029cm', con, 'wkb_geometry').to_crs(repro_crs)
flood_l2 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_121cm', con, 'wkb_geometry').to_crs(repro_crs)
flood_l3 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_271cm', con, 'wkb_geometry').to_crs(repro_crs)

### Flood 3

In [ ]:
# Dissolving by province
flood_dissolved = flood_l3.dissolve(by='deeperthan')
flood_dissolved_clean = flood_dissolved.set_index('id')[['wkb_geometry']]

In [ ]:
# Save to PostGIS
flood_dissolved_clean.to_postgis("afg_fldzonea_100k_ncia_v2_271cm_dissolve_all_v2", con)

### Flood 2

In [ ]:
# Dissolving by province
flood_dissolved = flood_l2.dissolve(by='deeperthan')
flood_dissolved_clean = flood_dissolved.set_index('id')[['wkb_geometry']]

In [ ]:
# Save to PostGIS
flood_dissolved_clean.to_postgis("afg_fldzonea_100k_ncia_v2_121cm_dissolve_all_v2", con)

### Flood 1

In [ ]:
# Dissolving by province
flood_dissolved = flood_l1.dissolve(by='deeperthan')
flood_dissolved_clean = flood_dissolved.set_index('id')[['wkb_geometry']]

In [ ]:
# Save to PostGIS
flood_dissolved_clean.to_postgis("afg_fldzonea_100k_ncia_v2_029cm_dissolve_all_v2", con)